In [69]:
%%time
import boto3
import re
import numpy as np
import pandas as pd
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker import image_uris
import cv2
import urllib.request as ur
from PIL import Image
import s3fs

role = get_execution_role()

training_image = image_uris.retrieve(
    region=boto3.Session().region_name, framework="image-classification"
)

CPU times: user 171 ms, sys: 5.23 ms, total: 176 ms
Wall time: 420 ms


In [96]:
import pathlib

def display_image(file_path):
    im = Image.open(file_path)
    im.show()
    
def get_image_size(file_path):
    im = Image.open(file_path)
    return im.size
    
min_size = (1000,1000)
max_size = (31,31) # which is the min_size
for directory in os.listdir("./train_data"):
    for file in os.scandir(f"./train_data/{directory}"):
    #     display_image(file.path)
        file_ext = pathlib.Path(file.path).suffix
        if file_ext == ".jpg":
            curr_size = get_image_size(file.path)
            if curr_size < min_size:
                min_size = curr_size
                
            if curr_size > max_size:
                max_size = curr_size
        
print(min_size)
print(max_size)

(31, 31)
(216, 216)


In [98]:
def resize():
    for directory in os.listdir("./train_data"):
        for file in os.scandir(f"./train_data/{directory}"):
            file_ext = pathlib.Path(file.path).suffix
            if file_ext == ".jpg":
                im = Image.open(file.path)
                f, e = os.path.splitext(file.path)
                imResize = im.resize((128,128), Image.ANTIALIAS)
                imResize.save(f + ' resized.jpg', 'JPEG', quality=90)
resize()

In [100]:
for directory in os.listdir("./train_data"):
        for file in os.scandir(f"./train_data/{directory}"):
            file_ext = pathlib.Path(file.path).suffix
            if file_ext == ".jpg":
                if file.path.count("resized") == 2:
                    os.remove(file.path)

In [ ]:
for directory in os.listdir(src_dir):
    for file in os.scandir(f"./{src_dir}/{directory}"):
        file_ext = pathlib.Path(file.path).suffix
        if file_ext == ".jpg" and "resized" in file.path:
            file_directory = os.path.join(dest_dir, directory)
            if not os.path.exists(file_directory):
                os.mkdir(file_directory)
            original_filepath = os.path.join(src_dir, directory, pathlib.Path(file.name))
#             print(original_filepath)
            shutil.copy2(original_filepath, file_directory)

In [ ]:
import shutil
import os
 
# path to source directory
src_dir = './train_data'
 
# path to destination directory
dest_dir = './train_data_resized'

In [ ]:
"""
we're splitting age groups into 5 buckets: 
0: 19-30
1: 31-40
2: 41-50
3: 51-60
4: 61-70
"""


bucket_0_folder = '0'
bucket_1_folder = '1'
bucket_2_folder = '2'
bucket_3_folder = '3'
bucket_4_folder = '4'

label_match_dict = {
    bucket_0_folder: [19, 30],
    bucket_1_folder: [31, 40],
    bucket_2_folder: [41, 50],
    bucket_3_folder: [51, 60],
    bucket_4_folder: [61, 70]
} 

def get_age_bucket(age): 
    if 19 < age <= 30:
        return bucket_0_folder
        
    if 30 < age <= 40:
        return bucket_1_folder
        
    if 40 < age <= 50:
        return bucket_2_folder
    
    if 50 < age <= 60:
        return bucket_3_folder
    
    return bucket_4_folder

bucket_destination_dir = "./train_buckets" 
os.mkdir(bucket_destination_dir)
for new_folder in label_match_dict:
    os.mkdir(os.path.join(bucket_destination_dir, new_folder))
    
for directory in os.listdir(dest_dir):
    for file in os.scandir(f"./{dest_dir}/{directory}"):
        file_ext = pathlib.Path(file.path).suffix
        if file_ext == ".jpg" and "resized" in file.path:
            
            folder = get_age_bucket(int(directory))
            new_filename = directory+str(pathlib.Path(file.name))
            file_destination = os.path.join(bucket_destination_dir, folder, new_filename)
            original_filepath = os.path.join(dest_dir, directory, pathlib.Path(file.name))
            shutil.copy2(original_filepath, file_destination)